In [43]:
def get_linguistic_data(url_input):
    """This function's argument is a URL of a wikipedia page, the return
       is a dictionary with the key being the language name, 
       the value is a dictionary, with the key being the name of the metadata field,
       and the value being the corresponding information of the language"""
    import urllib3
    from bs4 import BeautifulSoup
    import re
    import numpy as np

    http = urllib3.PoolManager()
    url = url_input
    response = http.request("GET", url)
    html_doc = response.data
    soup = BeautifulSoup(html_doc,features="html.parser")

    match_phrases = ["distribution","classification","proto","subdivisions","ethnicity",
                     "early","speakers","native","family","population","writing","extinct",
                     "region","dialects","era","standard","revival","reconstruction",
                    "reconstructed","reconstructions"]
    
    match_phrases_1 = ["reconstruction","reconstructed","reconstructions"]
    
    for i in soup.body.find_all("tbody"):
        try:
            if i.tr.th["colspan"] == "2" and "infobox" in i.parent.get("class"):
                table = i
                break
        except:
            pass
    dictionary = {}
    delimiter = '###' 
    
    for line_break in table.findAll('br'):  
        line_break.replaceWith(delimiter)
    dictionary["English Name"] = re.sub("\[([a-zA-Z]| |\d)+\]|^vte|vte?","",(" ".join(table.tr.th.get_text().split(delimiter))))
    dictionary["URL"] = url_input
    
    i = 0
    for j in table.find_all("th"):
        if re.compile("writing",re.I).search(j.get_text()) != None:
            i += 1
    if i >= 1:
        dictionary["Name in Local Language"] = []
    else:
        dictionary["Other Names"] = []
        
    for tag in table.find_all("tr")[1:3]:
        if tag.find_all("th") == [] and tag.find_all("td") != []:
            if tag.find_all(["li","i","span"]) == []:
                i = 0
                for j in table.find_all("th"):
                    if re.compile("writing",re.I).search(j.get_text()) != None:
                        i += 1
                if i >= 1:
                    text_3 = " ".join(tag.get_text().split(delimiter))
                    text_4 = re.sub("\\u200e","",text_3)
                    if text_4 not in dictionary["Name in Local Language"] and text_4 != "" and len(text_4) <= 40:
                        dictionary["Name in Local Language"].append(text_4)
                else:
                    text_3 = " ".join(tag.get_text().split(delimiter))
                    text_4 = re.sub("\\u200e","",text_3)
                    if text_4 not in dictionary["Other Names"] and text_4 != "" and len(text_4) <= 40:
                        dictionary["Other Names"].append(text_4)
            else:
                i = 0
                for j in table.find_all("th"):
                    if re.compile("writing",re.I).search(j.get_text()) != None:
                        i += 1
                if i >= 1:
                    for n in tag.find_all(["li","i","span"]):
                        list_4 = []
                        m = n.find_next_sibling()
                        if m != None:
                            while m.name != "i":
                                if m.name != "i" and m.name != "span":
                                    text_1 = "".join(re.split('\\n|c.\\u2009|\\xa0|'+delimiter,m.get_text()))
                                    text_1 = " ".join(re.split(re.compile(r"\[([a-zA-Z]| |\d)+\]"), text_1)).strip(" ")
                                    if text_1 not in list_4 and text_1 != "" and len(text_1) <= 40:
                                        list_4.append(text_1)
                                else:
                                    break
                                if m.find_next_sibling() != None:
                                    m = m.find_next_sibling()
                                else:
                                    break
                        text = " ".join(n.get_text().split(delimiter))+" "+" ".join(list_4)
                        text = " ".join(re.split(re.compile(r"\[([a-zA-Z]| |\d)+\]"), text)).strip()
                        text = re.sub("\\n|\\u200e","",text)
                        if text not in dictionary["Name in Local Language"] and text != "" and len(text) <= 40:
                            dictionary["Name in Local Language"].append(text)
                else:
                    for n in tag.find_all(["li","i","span"]):
                        list_4 = []
                        m = n.find_next_sibling()
                        if m != None:
                            while m.name != "i" and m.name != "span":
                                if m.name != "i" and m.name != "span":
                                    text_1 = "".join(re.split('\\n|c.\\u2009|\\xa0|'+delimiter,m.find_next_sibling().get_text()))
                                    text_1 = " ".join(re.split(re.compile(r"\\n|\[([a-zA-Z]| |\d)+\]"), text_1)).strip()
                                    if text_1 not in list_4 and text_1 != "" and len(text_1) <= 40:
                                        list_4.append(text_1)
                                else:
                                    break
                                if m.find_next_sibling() != None:
                                    m = m.find_next_sibling()
                                else:
                                    break
                        text = " ".join(n.get_text().split(delimiter))+" "+" ".join(list_4)
                        text = " ".join(re.split(re.compile(r"\\n|\[([a-zA-Z]| |\d)+\]"), text)).strip()
                        text = re.sub("\\n|\\u200e","",text)
                        if text not in dictionary["Other Names"] and text != "" and len(text) <= 40:
                            dictionary["Other Names"].append(text)

    for phrase_1 in match_phrases_1:
        if re.compile(r""+phrase_1,re.I).search(table.get_text()) != None:
            dictionary["Status"] = "Reconstructed Language"
        else:
            dictionary["Status"] = "Normal"
            
    for tag in table.find_all("th"):
        for phrase in match_phrases:
            if re.compile(r""+phrase,re.I).search(tag.get_text()) != None:
                if tag.parent.td != None:
                    if tag.parent.td.find_all("li") == []:
                        if re.compile(r"native",re.I).search(tag.get_text()) == None:
                            key = " ".join(re.split('\\n|c.\\u2009|\\xa0|'+delimiter,tag.get_text()))
                            value = [x for x in re.split('\\n|c.\\u2009|\\xa0|'+delimiter, tag.parent.td.get_text()) if x != ""]
                            key = " ".join(re.split(re.compile(r"\\n|\[([a-zA-Z]| |\d)+\]"), key))
                            value = [" ".join(re.split(re.compile(r"\\n|\[([a-zA-Z]| |\d)+\]"), i)).strip(" ") for i in value]
                            if value not in list(dictionary.values()) and value not in list(dictionary.keys()) and value != "" and key not in list(dictionary.values()) and key not in list(dictionary.keys()) and key != "":
                                dictionary[key] = value      
                        else:
                            key = " ".join(re.split('\\n|c.\\u2009|\\xa0|'+delimiter,tag.get_text()))
                            value = [x for x in re.split('\\n|c.\\u2009|\\xa0|'+delimiter, tag.parent.td.get_text()) if x != ""]
                            key = " ".join(re.split(re.compile(r"\\n|\[([a-zA-Z]| |\d)+\]"), key))
                            value = " ".join([" ".join(re.split(re.compile(r"\\n|\[([a-zA-Z]| |\d)+\]"), i)).strip(" ") for i in value]).strip(" | ")
                            if value not in list(dictionary.values()) and value not in list(dictionary.keys()) and value != "" and key not in list(dictionary.values()) and key not in list(dictionary.keys()) and key != "":
                                dictionary[key] = value
                    elif tag.parent.td.find("li").find_next_sibling("li") != None:
                        list_1 = []
                        key = " ".join(re.split('\\n|c.\\u2009|\\xa0|'+delimiter,tag.get_text()))
                        for i in tag.parent.td.find_all("li"):
                            value = [x for x in re.split('\\n|c.\\u2009|\\xa0|'+delimiter, i.get_text()) if x != ""]
                            value = [" ".join(re.split(re.compile(r"\\n|\[([a-zA-Z]| |\d)+\]"), i)).strip(" ") for i in value]
                            value = [" ".join(re.split(re.compile(r"\\n|\[([a-zA-Z]| |\d)+\]"), i)).strip(" ") for i in value]
                            value = " ".join(value)
                            try: 
                                if re.compile(re.escape("†|+"),re.I).search(i.get_text()) != None:
                                    value += " †"
                            except:
                                pass
                            try: 
                                if re.compile(re.escape("?"),re.I).search(i.get_text()) != None:
                                    value = "(?) "+value
                            except:
                                pass
                            if "?" in value:
                                value = re.sub("\(\?\) \(\?\)", "(?)",value)
                                value = value.rstrip("(?)").strip(" ")
                            list_1.append(value)
                            list_1 = sorted(list_1, key=len)
                            list_3 = []
                            for i in list_1:
                                b = 0
                                for j in list_3:
                                    if j in i:
                                        b += 1
                                if b == 0:
                                    list_3.append(i)
                        value_2 = [re.sub("\\n|\[([a-zA-Z]| |\d)+\]|^vte|vte?","",i) for i in list_3]
                        value = value_2
                        if value not in list(dictionary.values()) and value not in list(dictionary.keys()) and value != "" and key not in list(dictionary.values()) and key not in list(dictionary.keys()) and key != "":
                            dictionary[key] = value
                    else:
                        list_1 = []
                        key = " ".join(re.split('\\n|c.\\u2009|\\xa0|'+delimiter,tag.get_text()))
                        for i in tag.parent.td.find_all(["li","a","b"]):
                            value = [x for x in re.split('\\n|c.\\u2009|\\xa0|'+delimiter, i.find(text=True)) if x != ""]
                            value = [" ".join(re.split(re.compile(r"\\n|\[([a-zA-Z]| |\d)+\]"), i)).strip(" ") for i in value]
                            value = " ".join(value)
                            try: 
                                if re.compile(re.escape("†|+"),re.I).search(i.parent.find(text=True)) != None:
                                    value += " †"
                            except:
                                pass
                            try: 
                                if re.compile(re.escape("?"),re.I).search(i.parent.find(text=True)) != None:
                                    value = "(?) "+value
                            except:
                                pass
                            value = re.sub("\(\?\) \(\?\)", "(?)",value)
                            value = re.sub("citation needed", " ",value)
                            value = re.sub("\\n|\[([a-zA-Z]| |\d)+\]", "",value)
                            list_1.append(value)
                        value_1 = [i for i in list_1 if i != "" and re.match(r'^.*[a-zA-Z]+.*$', i) != None]
                        value_2 = [re.sub("^vte|vte?","",i) for i in value_1]
                        value = []
                        for i in value_2:
                            if i not in value:
                                value.append(i)
                        if value not in list(dictionary.values()) and value not in list(dictionary.keys()) and value != "" and key not in list(dictionary.values()) and key not in list(dictionary.keys()) and key != "":
                            dictionary[key] = value
                            
    for tag in table.find_all("td"):
        try:
            if tag.div.find_all("div") != []:
                if len(tag.div.find_all("div")) > 1:
                    if tag.div.find_all("p") == []:
                        list_1 = []
                        for i in tag.div.find_all("div"):
                            text = "".join(re.split('\\n|c.\\u2009|\\xa0|'+delimiter,i.get_text()))
                            if len(text) <= 45:
                                list_1.append(text)
                        if np.mean(list(map(lambda x: len(x),list_1))) < 35:
                            dictionary["Sub Languages"] = list_1
                    else:
                        dict_1 = {}
                        for i in tag.div.find_all("p"):
                            list_2 = []
                            j = i.find_next_sibling()
                            while j.name != "p":
                                if j.name == "div":
                                    list_2.append("".join(re.split('\\n|c.\\u2009|\\xa0|'+delimiter,j.get_text())))
                                if j.find_next_sibling() != None:
                                    j = j.find_next_sibling()
                                else:
                                    break
                            list_2 = [x for x in list_2 if x != ""]
                            if np.mean(list(map(lambda x: len(x),list_2))) < 35:
                                dict_1["".join(re.split('\\n|c.\\u2009|\\xa0|'+delimiter,i.get_text()))] = list_2
                                dictionary["Sub Languages"] = dict_1  
        except:
            pass
    list_4 = ["Language family","Early form","Linguistic classification","Dialect","Subdivision","Proto"]
    url_list = []
    for tag in table.find_all("th"):
        for phrase in list_4:
            if re.compile(r""+phrase,re.I).search(tag.get_text()) != None:
                if tag.parent.td != None:
                    if tag.parent.td.find_all("li") == []:
                        key = " ".join(re.split('\\n|c.\\u2009|\\xa0|'+delimiter,tag.get_text()))
                        list_3 = []
                        list_3.append(re.sub("\[([a-zA-Z]| |\d)+\]|^vte|vte?|\\n|c.\\u2009|\\xa0|","",tag.parent.td.get_text()))
                        dictionary[key+" corrected"]=list_3
                    else:
                        key = " ".join(re.split('\\n|c.\\u2009|\\xa0|'+delimiter,tag.get_text()))
                        list_3 = []
                        if tag.parent.td.find_all("a",recursive=False) != []:
                            url_1 = "https://en.wikipedia.org"+tag.parent.td.find("a",recursive=False).get("href")
                            list_3.append(\
                                          re.sub("\[([a-zA-Z]| |\d)+\]|^vte|vte?|\\n|c.\\u2009|\\xa0|","",tag.parent.td.find("a",recursive=False).find(text=True, recursive=False)))
                            url_list.append(url_1)
                        if tag.parent.td.find_all("div",recursive=False) != []:
                            i = tag.parent.td.find("div",recursive=False)
                            if i.find_all("a",recursive=False) == []:
                                if i.find_all(["b","span","i"],recursive=False)==[]:
                                    value = i.find(text=True)
                                    value = re.sub('\\n|c.\\u2009|\\xa0|'+delimiter,"", value)
                                    value = re.sub('\(.*\)',"", value)
                                    value = re.sub(re.compile(r"\\n|\[([a-zA-Z]| |\d)+\]|^vte|vte?"),"", value).strip(" ")
                                    list_3.append(value)
                                else:
                                    if i.find(["b","span","i"],recursive=False).find_all("a") == []:
                                        value = i.find(["b","span","i"],recursive=False).find(text=True)
                                        value = re.sub('\\n|c.\\u2009|\\xa0|'+delimiter,"", value)
                                        value = re.sub('\(.*\)',"", value)
                                        value = re.sub(re.compile(r"\\n|\[([a-zA-Z]| |\d)+\]|^vte|vte?"),"", value).strip(" ")
                                        list_3.append(value)
                                    else:
                                        if "not exist" in i.find(["b","span","i"],recursive=False).find("a").get('title'):
                                            list_3.append(i.find(["b","span","i"],recursive=False).find("a").get_text())
                                        else:
                                            url_1 = "https://en.wikipedia.org"+i.find(["b","span","i"],recursive=False).find("a").get('href')
                                            response_1 = http.request("GET", url_1)
                                            html_doc_1 = response_1.data
                                            soup_1 = BeautifulSoup(html_doc_1,features="html.parser")
                                            for i in soup_1.body.find_all("tbody"):
                                                try:
                                                    if i.tr.th["colspan"] == "2" and "infobox" in i.parent.get("class"):
                                                        table_1 = i
                                                        name_1 = re.sub("\\n|\[([a-zA-Z]| |\d)+\]|^vte|vte?","",(" ".join(table_1.tr.th.get_text().split(delimiter))))
                                                        url_list.append(url_1)
                                                        list_3.append(name_1)
                                                        break
                                                except:
                                                    pass        
                            else:
                                if "not exist" in i.find("a",recursive=False).get('title'):
                                    list_3.append(i.find("a",recursive=False).get_text())
                                else:
                                    url_1 = "https://en.wikipedia.org"+i.find("a",recursive=False).get('href')
                                    response_1 = http.request("GET", url_1)
                                    html_doc_1 = response_1.data
                                    soup_1 = BeautifulSoup(html_doc_1,features="html.parser")
                                    for i in soup_1.body.find_all("tbody"):
                                        try:
                                            if i.tr.th["colspan"] == "2" and "infobox" in i.parent.get("class"):
                                                table_1 = i
                                                name_1 = " ".join(table_1.tr.th.get_text().split(delimiter))
                                                name_1 = re.sub("\\n|^vte|vte?","",name_1)
                                                url_list.append(url_1)
                                                list_3.append(name_1)
                                                break
                                        except:
                                            pass
                        for i in tag.parent.td.find_all("li"):
                            if i.find_all("a",recursive=False)==[]:
                                if i.find_all(["b","span","i"],recursive=False)==[]:
                                    value = i.find(text=True, recursive=False)
                                    value = re.sub('\\n|c.\\u2009|\\xa0|'+delimiter,"", value)
                                    value = re.sub('\(.*\)',"", value)
                                    value = re.sub(re.compile(r"\\n|\[([a-zA-Z]| |\d)+\]|^vte|vte?"),"", value).strip(" ")
                                    list_3.append(value)
                                else:
                                    if i.find(["b","span","i"],recursive=False).find_all("a") == []:
                                        value = i.find(["b","span","i"],recursive=False).find(text=True)
                                        value = re.sub('\\n|c.\\u2009|\\xa0|'+delimiter,"", value)
                                        value = re.sub('\(.*\)',"", value)
                                        value = re.sub(re.compile(r"\\n|\[([a-zA-Z]| |\d)+\]|^vte|vte?"),"", value).strip(" ")
                                        list_3.append(value)
                                    else:
                                        if "not exist" in i.find(["b","span","i"],recursive=False).find("a").get('title'):
                                            list_3.append(i.find(["b","span","i"],recursive=False).find("a").get_text())
                                        else:
                                            url_1 = "https://en.wikipedia.org"+i.find(["b","span","i"],recursive=False).find("a",recursive=False).get('href')
                                            response_1 = http.request("GET", url_1)
                                            html_doc_1 = response_1.data
                                            soup_1 = BeautifulSoup(html_doc_1,features="html.parser")
                                            for i in soup_1.body.find_all("tbody"):
                                                try:
                                                    if i.tr.th["colspan"] == "2" and "infobox" in i.parent.get("class"):
                                                        table_1 = i
                                                        name_1 = re.sub("\\n|\[([a-zA-Z]| |\d)+\]|^vte|vte?","",(" ".join(table_1.tr.th.get_text().split(delimiter))))
                                                        url_list.append(url_1)
                                                        list_3.append(name_1)
                                                        break
                                                except:
                                                    pass               
                            else:
                                if "not exist" in i.find("a",recursive=False).get('title'):
                                    list_3.append(i.find("a",recursive=False).get_text())
                                else:
                                    url_1 = "https://en.wikipedia.org"+i.find("a",recursive=False).get('href')
                                    response_1 = http.request("GET", url_1)
                                    html_doc_1 = response_1.data
                                    soup_1 = BeautifulSoup(html_doc_1,features="html.parser")
                                    for i in soup_1.body.find_all("tbody"):
                                        try:
                                            if i.tr.th["colspan"] == "2" and "infobox" in i.parent.get("class"):
                                                table_1 = i
                                                name_1 = re.sub("\\n|\[([a-zA-Z]| |\d)+\]|^vte|vte?","",(" ".join(table_1.tr.th.get_text().split(delimiter))))
                                                url_list.append(url_1)
                                                list_3.append(name_1)
                                                break
                                        except:
                                            pass                      
                        dictionary[key+" corrected"]=list_3
    dictionary["Related URLs"]=url_list

    dictionary = {dictionary["English Name"]:dictionary}
    return dictionary

In [35]:
def get_related_url(url_input):
    """This function's argument is a URL of a wikipedia page, the return is a list of URLs,
    which link to the related wikipedia pages of the inputed wikipedia page URL. These 
    related URLs are scraped from the table in the upper right corner of the wikipedia page 
    and the sections such as "sub-groups", "sub-language", "classification" in the main body"""
    import urllib3
    from bs4 import BeautifulSoup
    import re

    http = urllib3.PoolManager()
    url = url_input
    response = http.request("GET", url)
    html_doc = response.data
    soup = BeautifulSoup(html_doc,features="html.parser")

    match_phrases = ["classification","proto","subdivisions",
                     "early","family","dialects","standard","reconstruction",
                    "reconstructed","reconstructions"]
    match_phrases_1 = ["classification","subdivisions","subgrouping","dialects",
                      "subclassification","languages","ancest","branch"]
                     
    for i in soup.body.find_all("tbody"):
        try:
            if i.tr.th["colspan"] == "2":
                table = i
                break
        except:
            pass
        
    delimiter = '###' 
    for line_break in table.findAll('br'):  
        line_break.replaceWith(delimiter)

    url_list = []
    for tag in table.find_all("th"):
        for phrase in match_phrases:
            if re.compile(r""+phrase,re.I).search(tag.get_text()) != None:
                if tag.parent.td != None:
                    if tag.parent.td.find_all("li") == []:# no list in tag
                        if re.compile(r"native",re.I).search(tag.get_text()) == None:
                            if tag.parent.td.find_all("a") != []:
                                for i in tag.parent.td.find_all("a"):
                                    url_list.append(i.get('href'))
                        else:
                            if tag.parent.td.find_all("a") != []:
                                for i in tag.parent.td.find_all("a"):
                                    url_list.append(i.get('href'))
                    elif tag.parent.td.find("li").find_next_sibling("li") != None: # paralleled lists in a division
                        for i in tag.parent.td.find_all("li"):
                            if i.find_all("a") != []:
                                for j in i.find_all("a"):
                                    url_list.append(j.get('href'))
                    else: # nested list within list
                        for i in tag.parent.td.find_all(["li","a","b"]):
                            if i.name == "a":
                                url_list.append(i.get('href'))
                            elif i.find_all("a") != []:
                                for j in i.find_all("a"):
                                    url_list.append(j.get('href'))
    for tag in table.find_all("td"):
        try:
            if tag.div.find_all("div") != []:
                if len(tag.div.find_all("div")) > 1:
                    for i in tag.div.find_all("div"):
                        if tag.find_all("a") != []:
                            for j in i.find_all("a"):
                                url_list.append(j.get('href'))
        except:
            pass
        
    for i in soup.body.find_all("tbody"):
        try:
            if i.tr.th["colspan"] == "2":
                table = i
                break
        except:
            pass
    if soup.find_all("h2") != []:
        for n in soup.find_all("h2"):
            i = 0
            for phrase in match_phrases_1:
                if re.compile(r""+phrase,re.I).search(n.get_text()) != None:
                    i += 1
            if i >= 1:
                tag = n.find_next_sibling()
                while tag.name != "h2":
                    if tag.find_all("a") != []:
                        for k in tag.find_all("a"):
                            url_list.append(k.get('href'))
                    if tag.find_next_sibling() != None:
                        tag = tag.find_next_sibling()
                    else:
                        break
            
    unwanted_list = ['/wiki/First_language','/wiki/Second_language','/wiki/Language_family']
    url_list_2 = []
    for i in url_list:
        if i != None:
            if i not in url_list_2 and i not in unwanted_list and i.startswith("/wiki/"):
                url_list_2.append(i)
    url_list = ["https://en.wikipedia.org"+i for i in url_list_2]
    return url_list

In [36]:
def Merge(dict1, dict2):
    """This function's argument are two dictionaries, the return is the merged dictionary 
    from these two dictionaries"""
    res = {**dict1, **dict2}
    return res

In [37]:
def check_language_family(language_family_list,url_input):
    """This function checks whether the wikipedia page of the inputed URL is a wikipedia page of language, and
    this language belongs to the specified language family. This function excludes wikipedia pages related to history,
    geography, people etc., only keeps pages on language, and excludes language that do not belong to the specified 
    language family, thus keep the results clean"""
    import urllib3
    from bs4 import BeautifulSoup
    import re

    http = urllib3.PoolManager()
    url = url_input
    response = http.request("GET", url)
    html_doc = response.data
    soup = BeautifulSoup(html_doc,features="html.parser")
    
    for i in soup.body.find_all("tbody"):
        try:
            if i.tr.th["colspan"] == "2" and "infobox" in i.parent.get("class"):
                table = i
                break
        except:
            pass
    a = 0
    text = table.get_text()
    text_2 = ", ".join([i.get_text() for i in table.find_all("th")])
    if language_family_list == ["Altaic"]:
        language_family_list = ["Altaic","Turkic","Mongolic","Tungusic","Koreanic","Japonic","Ainu"]
        for i in language_family_list:
            if re.compile(r""+i,re.I).search(text) != None:
                a += 1
        if a >= 1 and re.compile("language|linguistic|reconstruct",re.I).search(text_2) != None:
            return True
        else:
            return False
    elif language_family_list == ["Afroasiatic"]:
        language_family_list = ["Afroasiatic","Afro-Asiatic","Asiatic","Afro-"]
        for i in language_family_list:
            if re.compile(r""+i,re.I).search(text) != None:
                a += 1
        if a >= 1 and re.compile("language|linguistic|reconstruct",re.I).search(text_2) != None:
            return True
        else:
            return False
    else:
        for i in language_family_list:
            if re.compile(r""+i,re.I).search(text) != None:
                a += 1
        if a >= 1 and re.compile("language|linguistic|reconstruct",re.I).search(text_2) != None:
            return True
        else:
            return False

In [51]:
def get_language_family_data(language_family,number,additional_start_url_list=[]):
    """This function will search through the wikipedia pages and scrape the web and get the languages' information
    and store the information in one dictionary. It starts from the starting wikipedia URL, get related URLs, add these 
    URLs to the unsearched_url_list. It will search every URL in the unsearched_url_list, put the searched URL into 
    searched_url_list. If the seearched URL is a language and contains useful and related information, the URL will be 
    put into found_url_list. If the URL has been searched, it will not be searched again. The return a a dictionary with 
    the names of the language being the keys, and scraped metadata being the values. 
    """
    import time
    from collections import deque

    start_url = "https://en.wikipedia.org/wiki/"+language_family+"_languages"
    unsearched_url_list = [start_url]+additional_start_url_list
    searched_url_list = []
    found_url_list = []
    languages_dict = {}
    deque = deque(unsearched_url_list)
    
    while len(found_url_list) <= number and len(deque) > 0:
        if len(deque) <= 5:
            for i in unsearched_url_list[unsearched_url_list.index(deque[-1])+1:]:
                deque.append(i)
        url = deque.popleft()
        try:
            language_family_list = [language_family]
            if check_language_family(language_family_list,url):
                new_language_dict = get_linguistic_data(url)
                for key in new_language_dict.keys():
                    if key not in languages_dict.keys():
                        languages_dict = Merge(languages_dict,new_language_dict)
                        print("Now scraping from     "+url)
                        print("New linguistic unit is     "+key)
                        found_url_list.append(url)
                        searched_url_list.append(url)
                        try:
                            print("      found_url_list index: "+str(found_url_list.index(url)))
                            print("      searched_url_list index: "+str(searched_url_list.index(url)))
                            print("      unsearched_url_list index: "+str(unsearched_url_list.index(url)))
                        except:
                            pass
                    else:
                        print("         Have already looked at this one   "+url)
                        try:
                            print("               found_url_list index: "+str(found_url_list.index(url)))
                        except:
                            pass
                        try:
                            print("               searched_url_list index: "+str(searched_url_list.index(url)))
                        except:
                            pass
                        try:
                            print("               unsearched_url_list index: "+str(unsearched_url_list.index(url)))
                        except:
                            pass
                for url_1 in get_related_url(url):
                    new_url = url_1
                    if new_url not in unsearched_url_list and new_url not in searched_url_list:
                        deque.append(new_url)
                    if new_url not in unsearched_url_list:
                        unsearched_url_list.append(new_url)
                print("                       deque length: "+str(len(deque)))
                print("                       found_url_list length: "+str(len(found_url_list)))
                print("                       searched_url_list length: "+str(len(searched_url_list)))
                print("                       unsearched_url_list length: "+str(len(unsearched_url_list)))
                print("                       "+str(len(languages_dict.keys()))+" languages in the dictionary")
                time.sleep(0)
            else:
                print("                  This one does not belong here   "+url)
                searched_url_list.append(url)
                try:
                    print("                         searched_url_list index: "+str(searched_url_list.index(url)))
                    print("                         unsearched_url_list index: "+str(unsearched_url_list.index(url)))
                except:
                    pass
        except:
            pass
    return languages_dict

In [6]:
def write_to_json(data,file_name):
    """write the nested list and dictionary into a json file"""
    import json
    data_json = json.dumps(data)
    with open(file_name, "w") as outfile:
        outfile.write(data_json)

In [7]:
def read_from_json(file_name):
    """read the  json file into a nested list and dictionary"""
    import json
    with open(file_name, "r",encoding="UTF-8") as outfile:
        read_data = outfile.read()
    return json.loads(read_data)

In [8]:
language_family_list = ["Indo-European","Sino-Tibetan","Austronesian","Uralic"
                   "Niger–Congo","Dravidian","Austroasiatic","Afroasiatic","Altaic"]

In [9]:
test_language_family_list = ["Indo-European","Sino-Tibetan","Altaic","Uralic","Afroasiatic"]

In [54]:
name = "Indo-European"
additional_start_url_list = get_related_url("https://en.wikipedia.org/wiki/List_of_Indo-European_languages")
write_to_json(get_language_family_data(name,5000,additional_start_url_list),name+" language data.json")

Now scraping from     https://en.wikipedia.org/wiki/Indo-European_languages
New linguistic unit is     Indo-European
      found_url_list index: 0
      searched_url_list index: 0
      unsearched_url_list index: 0
                       deque length: 4077
                       found_url_list length: 1
                       searched_url_list length: 1
                       unsearched_url_list length: 4078
                       1 languages in the dictionary
                  This one does not belong here   https://en.wikipedia.org/wiki/List_of_Afro-Asiatic_languages
                         searched_url_list index: 1
                         unsearched_url_list index: 2
                  This one does not belong here   https://en.wikipedia.org/wiki/List_of_Tungusic_languages
                         searched_url_list index: 2
                         unsearched_url_list index: 5
Now scraping from     https://en.wikipedia.org/wiki/Proto-Indo-European_language
New linguistic unit is  

In [55]:
name = "Sino-Tibetan"
additional_start_url_list = []
write_to_json(get_language_family_data(name,5000,additional_start_url_list),name+" language data.json")

Now scraping from     https://en.wikipedia.org/wiki/Sino-Tibetan_languages
New linguistic unit is     Sino-Tibetan
      found_url_list index: 0
      searched_url_list index: 0
      unsearched_url_list index: 0
                       deque length: 116
                       found_url_list length: 1
                       searched_url_list length: 1
                       unsearched_url_list length: 117
                       1 languages in the dictionary
Now scraping from     https://en.wikipedia.org/wiki/Proto-Sino-Tibetan_language
New linguistic unit is     Proto-Sino-Tibetan
      found_url_list index: 1
      searched_url_list index: 1
      unsearched_url_list index: 1
                       deque length: 116
                       found_url_list length: 2
                       searched_url_list length: 2
                       unsearched_url_list length: 118
                       2 languages in the dictionary
Now scraping from     https://en.wikipedia.org/wiki/Sinitic_languag

In [52]:
name = "Uralic"
additional_start_url_list = []
write_to_json(get_language_family_data(name,5000,additional_start_url_list),name+" language data.json")

Now scraping from     https://en.wikipedia.org/wiki/Uralic_languages
New linguistic unit is     Uralic
      found_url_list index: 0
      searched_url_list index: 0
      unsearched_url_list index: 0
                       deque length: 39
                       found_url_list length: 1
                       searched_url_list length: 1
                       unsearched_url_list length: 40
                       1 languages in the dictionary
Now scraping from     https://en.wikipedia.org/wiki/Proto-Uralic_language
New linguistic unit is     Proto-Uralic
      found_url_list index: 1
      searched_url_list index: 1
      unsearched_url_list index: 1
                       deque length: 40
                       found_url_list length: 2
                       searched_url_list length: 2
                       unsearched_url_list length: 42
                       2 languages in the dictionary
Now scraping from     https://en.wikipedia.org/wiki/Finnic_languages
New linguistic unit is    

In [53]:
name = "Altaic"
additional_start_url_list = []
write_to_json(get_language_family_data(name,5000,additional_start_url_list),name+" language data.json")

Now scraping from     https://en.wikipedia.org/wiki/Altaic_languages
New linguistic unit is     Altaic
      found_url_list index: 0
      searched_url_list index: 0
      unsearched_url_list index: 0
                       deque length: 7
                       found_url_list length: 1
                       searched_url_list length: 1
                       unsearched_url_list length: 8
                       1 languages in the dictionary
Now scraping from     https://en.wikipedia.org/wiki/Proto-Altaic_language
New linguistic unit is     Proto-Altaic
      found_url_list index: 1
      searched_url_list index: 1
      unsearched_url_list index: 1
                       deque length: 11
                       found_url_list length: 2
                       searched_url_list length: 2
                       unsearched_url_list length: 13
                       2 languages in the dictionary
Now scraping from     https://en.wikipedia.org/wiki/Turkic_languages
New linguistic unit is     T

In [56]:
name = "Afroasiatic"
additional_start_url_list = []
write_to_json(get_language_family_data(name,5000,additional_start_url_list),name+" language data.json")

Now scraping from     https://en.wikipedia.org/wiki/Afroasiatic_languages
New linguistic unit is     Afroasiatic
      found_url_list index: 0
      searched_url_list index: 0
      unsearched_url_list index: 0
                       deque length: 69
                       found_url_list length: 1
                       searched_url_list length: 1
                       unsearched_url_list length: 70
                       1 languages in the dictionary
Now scraping from     https://en.wikipedia.org/wiki/Proto-Afroasiatic_language
New linguistic unit is     Proto-Afroasiatic
      found_url_list index: 1
      searched_url_list index: 1
      unsearched_url_list index: 1
                       deque length: 72
                       found_url_list length: 2
                       searched_url_list length: 2
                       unsearched_url_list length: 74
                       2 languages in the dictionary
Now scraping from     https://en.wikipedia.org/wiki/Berber_languages
New li